In [86]:
import numpy as np
import numpy.linalg as linalg
import os
import librosa as lb
import soundfile as sf
import museval
from matplotlib import pyplot as plt
import itertools
import random
from tqdm import tqdm

## Frame-wise

In [2]:
def leastnormsoln(r, s):
    b = np.array([r-s]).T
    A = np.array([[s[1], s[2], s[3], 0, 0 ,0, 0, 0, 0, 0, 0, 0], 
                  [0, 0, 0, s[0], s[2], s[3], 0, 0, 0, 0, 0, 0], 
                  [0, 0, 0, 0, 0, 0, s[0], s[1], s[3], 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, s[0], s[1], s[2]]])
    soln, residual, rank, singular = linalg.lstsq(A, b, rcond=-1)
    return np.array([[1, soln[0][0], soln[1][0], soln[2][0]],
                     [soln[3][0], 1, soln[4][0], soln[5][0]],
                     [soln[6][0], soln[7][0], 1, soln[8][0]],
                     [soln[9][0], soln[10][0], soln[11][0], 1]])

In [3]:
def norm(A):
    forb = linalg.norm(A, ord='fro')
    l1 = linalg.norm(A, ord=1)
    return l1, forb

In [20]:
def framewise(R, S):
    lmda = []
    for i in range(R.shape[1]):
        r = R[:, i]
        s = S[:, i]
        lmda.append(leastnormsoln(r, s))
        
    l1, forb = [], []
    for i in lmda:
        l_1, forbn = norm(i)
        l1.append(l_1)
        forb.append(forbn)
        
    d_l1, d_forb = differenceNorm(lmda)
    
    return l1, forb, d_l1, d_forb

## Block-wise

In [55]:
def blockleastnormsoln(r, s):
    r = r.flatten()
    s = s.flatten()
    b = r-s
    
    val = int(r.shape[0]/4)
    s0 = s[0:val]
    s1 = s[val:2*val]
    s2 = s[2*val:3*val]
    s3 = s[3*val:4*val]
    
    z = np.zeros(val)

    c1 = np.hstack((s1, z, z, z))
    c2 = np.hstack((s2, z, z, z))
    c3 = np.hstack((s3, z, z, z))

    c4 = np.hstack((z, s0, z, z))
    c5 = np.hstack((z, s2, z, z))
    c6 = np.hstack((z, s3, z, z))

    c7 = np.hstack((z, z, s0, z))
    c8 = np.hstack((z, z, s1, z))
    c9 = np.hstack((z, z, s3, z))

    c10 = np.hstack((z, z, z, s0))
    c11 = np.hstack((z, z, z, s1))
    c12 = np.hstack((z, z, z, s2))
    
    A = np.stack((c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12), axis=1)
    
    soln, residual, rank, singular = linalg.lstsq(A, b, rcond=-1)
    
    return np.array([[1, soln[0], soln[1], soln[2]],
                     [soln[3], 1, soln[4], soln[5]],
                     [soln[6], soln[7], 1, soln[8]],
                     [soln[9], soln[10], soln[11], 1]])

In [56]:
def blockwise(R, S, time=0.5):

    block = int(time * fs)

    block_lamda = []
    for i in range(0, R.shape[1], block):
        r = R[: ,i:i+block]
        s = S[: ,i:i+block]
        block_lamda.append(blockleastnormsoln(r, s))
        
    l1, forb = [], []
    for i in block_lamda:
        l_1, forbn = norm(i)
        l1.append(l_1)
        forb.append(forbn)
        
    d_l1, d_forb = differenceNorm(block_lamda)
    
    return l1, forb, d_l1, d_forb

## Plotting 

In [84]:
def plot_time(y, norm, method, out_path):
    if method == 'frame':
        x = np.arange(len(y))/22050
        plt.plot(x, y)
        plt.title(norm + ' norm')
        plt.xlabel('seconds')
        plt.ylabel('norm')
        plt.savefig(out_path, dpi = 1000)
        plt.show()
    else:
        x = np.arange(len(y))/2
        plt.plot(x, y)
        plt.title(norm + ' norm')
        plt.xlabel('seconds')
        plt.ylabel('norm')
        plt.savefig(out_path, dpi = 1000)
        plt.show()


In [85]:
def plot_hist(x, norm, out_path):
    plt.hist(x)
    plt.title(norm + ' Histogram')
    plt.xlabel('Norm')
    plt.ylabel('frequency')
    plt.savefig(out_path, dpi = 1000)
    plt.show()


## Difference Norm

In [68]:
def differenceNorm(lmbda):
    random.shuffle(lmbda)
    lamda_comb = list(itertools.combinations(lmbda[:1000], 2))
    
    diff = []
    for everyblock in lamda_comb:
        diff.append(everyblock[0]-everyblock[1])
        
    l1, forb = [], []
    for i in diff:
        l_1, forbn = norm(i)
        l1.append(l_1)
        forb.append(forbn)
    
    return l1, forb

## Script

In [64]:
def obtain_norms(bleed_path, clean_path):
    
    bvocals, fs = lb.load(bleed_path+'vocals.wav')
    bbass, fs = lb.load(bleed_path+'bass.wav')
    bdrums, fs = lb.load(bleed_path+'drums.wav')
    bother, fs = lb.load(bleed_path+'other.wav')

    vocals, fs = lb.load(clean_path+'vocals.wav')
    bass, fs = lb.load(clean_path+'bass.wav')
    drums, fs = lb.load(clean_path+'drums.wav')
    other, fs = lb.load(clean_path+'other.wav')

    if len(bbass) > len(bass):
        n = len(bass)
    else:
        n = len(bbass)

    R = np.array([bvocals[:n], bbass[:n], bdrums[:n], bother[:n]])
    S = np.array([vocals[:n], bass[:n], drums[:n], other[:n]])
    
    l1_frame, forb_frame, diff_l1_frame, diff_forb_frame = framewise(R, S)
    l1_block, forb_block, diff_l1_block, diff_forb_block = blockwise(R, S, 0.5)
    
    frame = [l1_frame, forb_frame, diff_l1_frame, diff_forb_frame]
    block = [l1_block, forb_block, diff_l1_block, diff_forb_block]
    return frame, block

In [75]:
def obtain_plots(frame, block, folder, dest_path):
    l1_frame, forb_frame, diff_l1_frame, diff_forb_frame = frame[0], frame[1], frame[2], frame[3]
    l1_block, forb_block, diff_l1_block, diff_forb_block = block[0], block[1], block[2], block[3]
    
    out_path = dest_path+folder
    
    os.makedirs(out_path, exist_ok=True)
    
    #Frame-wise
    plot_time(l1_frame, norm='l1', method='frame', out_path=out_path+'/l1.png')
    plot_time(forb_frame, norm='forb', method='frame', out_path=out_path+'/forb.png')
    
    plot_hist(l1_frame, norm='l1', out_path=out_path+'/hist_l1.png')
    plot_hist(forb_frame, norm='forb', out_path=out_path+'/hist_forb.png')
    plot_hist(diff_l1_frame, norm='Difference l1', out_path=out_path+'/diffhist_l1.png')
    plot_hist(diff_forb_frame, norm='Difference forb', out_path=out_path+'/diffhist_forb.png')
    
    #Block-wise
    plot_time(l1_block, norm='l1', method='block', out_path=out_path+'/l1_block.png')
    plot_time(forb_block, norm='forb', method='block', out_path=out_path+'/forb_block.png')
    
    plot_hist(l1_block, norm='l1', out_path=out_path+'/hist_l1_block.png')
    plot_hist(forb_block, norm='forb', out_path=out_path+'/hist_forb_block.png')
    plot_hist(diff_l1_block, norm='Difference l1', out_path=out_path+'/diffhist_l1_block.png')
    plot_hist(diff_forb_block, norm='Difference forb', out_path=out_path+'/diffhist_forb_block.png')

## Driver Code

In [83]:
bleed_path = '/home/rajesh/Desktop/Datasets/musdb18hq_bleeded/train/'
clean_path = '/home/rajesh/Desktop/Datasets/musdb18hq/train/'
dest_path = '/home/rajesh/Desktop/Results/'

In [ ]:
files = sorted(os.listdir(bleed_path))

for i in tqdm(files[:10]):
    frame, block = obtain_norms(bleed_path+i+'/', clean_path+i+'/')
    obtain_plots(frame, block, i , dest_path)

# Generate PDF Document